In [16]:
import torch

import torch.nn    as nn
import torch.optim as optim
import networkx    as nx

from torch.utils.data       import random_split
from torch_geometric.utils  import convert
from torch_geometric.data   import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn     import GraphConv, Linear

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
n_epochs      = 1000
batch_size    = 128
learning_rate = 0.0001

# Number of diffusing and denoising steps, which can be different
n_diffusing_steps = 10
n_denoising_steps = 10

# Dropouts for node and edge models (work independently)
dropout_node = 0.2
dropout_edge = 0.2

# Target to generate new crystals
target = 'D'

input_folder  = '../MP/models'
target_folder = f'{input_folder}/{target}'
model_name    = f'{target_folder}/model.pt'

# Generation of graph database for training

Load the datasets, already standarized if possible.

In [3]:
labels_name         = f'{target_folder}/labels.pt'
dataset_name        = f'{target_folder}/dataset.pt'
dataset_name_std    = f'{target_folder}/standardized_dataset.pt'
parameters_name_std = f'{target_folder}/standardized_parameters.pt'  # Parameters for rescaling the predictions

try:    
    dataset    = torch.load(dataset_name_std)
    labels     = torch.load(labels_name)
    parameters = torch.load(parameters_name_std)

    # Assigning parameters accordingly
    target_mean, feat_mean, edge_mean, target_std, edge_std, feat_std, scale = parameters
    
    # Defining target factor
    target_factor = target_std / scale
except FileNotFoundError:
    dataset = torch.load(dataset_name)
    labels  = torch.load(labels_name)
    
    ### Santadirizing properties

    # Compute means and standard deviations

    target_list = torch.tensor([])
    edge_list   = torch.tensor([])

    for data in dataset:
        target_list = torch.cat((target_list, data.y),         0)
        edge_list   = torch.cat((edge_list,   data.edge_attr), 0)

    scale = 1e0

    target_mean = torch.mean(target_list)
    target_std  = torch.std(target_list)

    edge_mean = torch.mean(edge_list)
    edge_std  = torch.std(edge_list)

    target_factor = target_std / scale
    edge_factor   = edge_std   / scale

    # Update normalized values into the database

    for data in dataset:
        data.y         = (data.y         - target_mean) / target_factor
        data.edge_attr = (data.edge_attr - edge_mean)   / edge_factor

    # Same for the node features

    feat_mean = torch.tensor([])
    feat_std  = torch.tensor([])

    for feat_index in range(dataset[0].num_node_features):
        feat_list = torch.tensor([])

        for data in dataset:
            feat_list = torch.cat((feat_list, data.x[:, feat_index]), 0)

        feat_mean = torch.cat((feat_mean, torch.tensor([torch.mean(feat_list)])), 0)
        feat_std  = torch.cat((feat_std,  torch.tensor([torch.std(feat_list)])),  0)

        for data in dataset:
            data.x[:, feat_index] = (data.x[:, feat_index] - feat_mean[feat_index]) * scale / feat_std[feat_index]
    
    parameters = [target_mean, feat_mean, edge_mean, target_std, edge_std, feat_std, scale]
    
    torch.save(dataset,    dataset_name_std)
    torch.save(parameters, parameters_name_std)

# Generation of diffusing and denoising Markov chains

In [6]:
def get_alpha_t(t, T, s=1e-2):
    """Defines constant alpha at time-step t, given a parameter s < 0.5 (else alpha increases).
    
    Args:
        t (int):   time step (of diffusion or denoising) in which alpha is required.
        T (int):   total number of steps.
        s (float): parameter which controls the decay of alpha with t.
    
    Returns:
        alpha (float): parameter which controls the velocity of diffusion or denoising.
    """
    
    return torch.tensor((1 - 2 * s) * (1 - (t / T)**2) + s)


def get_random_graph(n_nodes, n_features, in_edge_index=None, n_edges=None):
    """Generates a random graph with specified number of nodes and features, and attributes. It is assumed
    that all parameters are normally distributed N(0, 1).
    
    Args:
        n_nodes       (int):   Number of nodes.
        n_features    (int):   Number of features for each node.
        in_edge_index (array): Positions of high-symmetry points in k-space (if None, they are randomized).
        n_edges       (int):   Number of edges, if edge_index is randomized (if None, it is randomized).
    
    Returns:
        graph (torch_geometric.data.data.Data): Graph structure with random node features and edge attributes.
    """
    
    if in_edge_index is None:  # Randomize edge indexes
        if n_edges is None:  # Randomize number of edges
            n_edges = torch.randint(low=50, high=101, size=(1,)).item()
        edge_index = torch.randn(2, n_edges)
    else:
        # Clone edge indexes
        edge_index = torch.clone(in_edge_index)
    
    # Get number of edges
    n_edges = torch.Tensor.size(edge_index)[1]
    
    # Generate random node features
    x = torch.randn(n_nodes, n_features)
    
    # Generate random edge attributes
    edge_attr = torch.randn(n_edges, 1)
    
    # Define graph with generated inputs
    graph = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
    return graph


def diffusion_step(graph_0, t, n_diffusing_steps):
    """Performs a forward step of a diffusive, Markov chain.
    
    Args:
        graph_0 (torch_geometric.data.data.Data): Graph which is to be diffused (step t-1).
    
    Returns:
        graph_t (torch_geometric.data.data.Data): Diffused graph (step t).
    """
    
    # Clone graph that we are diffusing (not extrictly necessary)
    graph_t = graph_0.clone()
    
    # Number of nodes and features in the graph
    n_nodes, n_features = torch.Tensor.size(graph_t.x)
    
    # Generate gaussian (normal) noise
    epsilon = get_random_graph(n_nodes, n_features, graph_t.edge_index)
    
    # Compute alpha_t
    alpha_t = get_alpha_t(t, n_diffusing_steps)
    
    # Forward pass
    graph_t.x         = torch.sqrt(alpha_t) * graph_t.x         + torch.sqrt(1 - alpha_t) * epsilon.x
    graph_t.edge_attr = torch.sqrt(alpha_t) * graph_t.edge_attr + torch.sqrt(1 - alpha_t) * epsilon.edge_attr
    return graph_t


def diffuse(graph_0, n_diffusing_steps):
    """Performs consecutive steps of diffusion in a reference graph.
    
    Args:
        graph_0           (torch_geometric.data.data.Data): Reference graph to be diffused (step t-1).
        n_diffusing_steps (int):                            Number of diffusive steps.
    
    Returns:
        graph_t (torch_geometric.data.data.Data): Graph with random node features and edge attributes (step t).
    """
    
    graph_t = graph_0.clone()
    for t in range(n_diffusing_steps):
        graph_t = diffusion_step(graph_t, t, n_diffusing_steps)
    return graph_t


def denoising_step(graph_t, epsilon, t, n_denoising_steps):
    """Performs a forward step of a denoising chain.
    
    Args:
        graph_t (torch_geometric.data.data.Data): Graph which is to be denoised (step t).
        epsilon (torch_geometric.data.data.Data): Predicted noise to substract.
    
    Returns:
        graph_0 (torch_geometric.data.data.Data): Denoised graph (step t-1).
    """
    
    # Clone graph that we are denoising (not extrictly necessary)
    graph_0 = graph_t.clone()
    
    # Compute alpha_t
    alpha_t = get_alpha_t(t, n_denoising_steps)
    
    # Backard pass
    graph_0.x         = graph_0.x         / torch.sqrt(alpha_t) - torch.sqrt((1 - alpha_t) / alpha_t) * epsilon.x
    graph_0.edge_attr = graph_0.edge_attr / torch.sqrt(alpha_t) - torch.sqrt((1 - alpha_t) / alpha_t) * epsilon.edge_attr
    return graph_0

# Generation of Graph Neural Network models

In [12]:
class nGCNN(torch.nn.Module):
    """Graph convolution neural network for the prediction of node embeddings.
    """
    
    def __init__(self, features_channels, pdropout):
        super(nGCNN, self).__init__()
        
        # Set random seed for reproducibility
        torch.manual_seed(12345)
        
        # Define graph convolution layers
        self.conv1 = GraphConv(features_channels, 512)
        self.conv2 = GraphConv(512, 512)
        
        # Define linear layers
        self.linconv = Linear(512, 16)
        self.lin     = Linear(16, 1)
        
        self.pdropout = pdropout

    def forward(self, x, edge_index, edge_attr, batch):
        ## CONVOLUTION
        
        # Apply graph convolution with ReLU activation function
        x = self.conv1(x, edge_index, edge_attr)
        x = x.relu()
        x = self.conv2(x, edge_index, edge_attr)
        return x


class eGCNN(nn.Module):
    """Graph convolution neural network for the prediction of edge attributes.
    """
    
    def __init__(self, features_channels, pdropout):
        super(eGCNN, self).__init__()

        self.linear1 = Linear(features_channels, 32)
        self.linear2 = Linear(32, features_channels)

        self.pdropout = pdropout

    def forward(self, x_i, x_j):
        # Dot product between node distances (?)
        x = x_i * x_j
        
        # Linear convolutions
        x = self.linear1(x)
        x = x.relu()
        
        # Dropout layer (only for training)
        x = F.dropout(x, p=self.pdropout, training=self.training)
        
        # Last linear convolution
        x = self.linear2(x)
        x = x.relu()
        return x

# Definition of train-test datasets

In [9]:
# torch.manual_seed(12345)

# Define the sizes of the train and test sets
train_size = int(0.8 * len(dataset))
test_size  = len(dataset) - train_size

# Use random_split() to generate train and test sets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of testing  graphs: {len(test_dataset)}')

Number of training graphs: 171
Number of testing  graphs: 43


# Training of the model

In [18]:
# Determine number of features in dataset
n_features = dataset[0].num_node_features

# Instantiate the models for nodes and edges
node_model = nGCNN(n_features, dropout_node).to(device)
edge_model = eGCNN(n_features, dropout_edge).to(device)
print('\nNode GCNN:')
print(node_model)
print('\nEdge GCNN:')
print(edge_model)


Nodes:
nGCNN(
  (conv1): GraphConv(5, 512)
  (conv2): GraphConv(512, 512)
  (linconv): Linear(512, 16, bias=True)
  (lin): Linear(16, 1, bias=True)
)

Edges:
eGCNN(
  (linear1): Linear(100, 64, bias=True)
  (linear2): Linear(64, 64, bias=True)
  (linear3): Linear(64, 100, bias=True)
)


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion_node = nn.MSELoss()
criterion_edge = nn.MSELoss()

# Training loop
for epoch in range(n_epochs):
    # Training
    
    
    train_loss = 0
    for graph in train_dataset:
        optimizer.zero_grad()
        
        # Diffuse the graph with some noise
        diffused_graph = diffuse(graph, n_diffusing_steps)
        
        # Denoise the diffused graph
        denoised_graph = diffused_graph.clone()
        for t in range(n_denoising_steps):
            # Perform a single forward pass for predicting node features
            out_x = node_model(diffused_graph.x, 
                               diffused_graph.edge_index,
                               diffused_graph.edge_attr)
            
            # Define edges
            # DEFINE x_i and x_j as connected nodes
            
            # Perform a single forward pass for predicting edge attributes
            out_attr = edge_model(x_i, 
                                  x_j)

            # Construct noise graph
            noise_graph = Data(x=out_x, edge_index=diffused_graph.edge_index, edge_attr=out_attr)

            # Denoise the graph with the predicted noise
            denoised_graph = denoising_step(diffused_graph, noise_graph, t, n_denoising_steps)

        # Calculate the loss for node features
        loss_node = criterion_node(graph.x, denoised_graph.x)

        # Calculate the loss for edge attributes
        loss_edge = criterion_edge(graph.edge_attr, denoised_graph.edge_attr)

        # Accumulate the total training loss
        loss = loss_node + loss_edge
        train_loss = loss.item()

        # Backpropagation and optimization step
        loss.backward()
        optimizer.step()
    
    # Compute the average train loss
    train_loss = train_loss / len(train_loader)
    
    print(f'Epoch: {epoch+1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')